In [1]:
DUNE_API_KEY = '/key/' #powerpool

### CoW Protocol Client

https://github.com/cowprotocol/dune-client

In [2]:
#pip install dune-client
#pip install python-dotenv

In [3]:
import dotenv
import os
import pandas as pd
import json

from dune_client.types import QueryParameter
from dune_client.client import DuneClient
from dune_client.query import QueryBase

In [4]:
query = QueryBase(
    name="Sample Query",
    query_id=347203,
#     params=[
#         QueryParameter.text_type(name="TextField", value="Word"),
#         QueryParameter.number_type(name="NumberField", value=3.1415926535),
#         QueryParameter.date_type(name="DateField", value="2022-05-04 00:00:00"),
#         QueryParameter.enum_type(name="ListField", value="Option 1"),
#     ],
)
print("Results available at", query.url())

Results available at https://dune.com/queries/347203


In [5]:
dotenv.load_dotenv()
dune = DuneClient(DUNE_API_KEY) #os.environ["DUNE_API_KEY"]

In [6]:
#dataframe
# data = dune.refresh_into_dataframe(query)
# data

In [7]:
#json
data2 = dune.refresh(query, performance = 'large')
data2 = data2.get_rows()
data2

2024-08-31 00:42:32,431 INFO dune_client.base_client executing 347203 on large cluster
2024-08-31 00:42:33,107 INFO dune_client.base_client waiting for query execution 01J6JKTVQK45TPG03KWZ1PDBV7 to complete: ExecutionState.PENDING (queue position: 2)


[{'CVP_balance': 17034449,
  'holder': '0xb258302c3f209491d604165549079680708581cc',
  'holder_name': 'Multisig'},
 {'CVP_balance': 9510562,
  'holder': '0x60842a6b44cca5eb7685d1135acf69b1336b28b3',
  'holder_name': 'Old PP Vesting'},
 {'CVP_balance': 9143920,
  'holder': '0x82c2b82e71998eb94be0a9f353e7a9eb7f74c603',
  'holder_name': 'Ecosystem Fund (Grants)'},
 {'CVP_balance': 7500000,
  'holder': '0x7cf2c758d73bf08d40a05813e6e03906bb97c90a',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 5000000,
  'holder': '0x5a20e71378bc770d1cd0b9a30d96bf911f023d1c',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 5000000,
  'holder': '0x836e76c340f969e5bab521f69ea26218a189bab0',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 4836246,
  'holder': '0x0d0707963952f2fba59dd06f2b425ace40b492fe',
  'holder_name': 'Gate.io 1'},
 {'CVP_balance': 3808230,
  'holder': '0x28c6c06298d514db089934071355e5743bf21d60',
  'holder_name': 'Binance 14'},
 {'CVP_balance': 3189842,
  'holder': '0xdfd5293d8e

### Python Requests

In [8]:
from requests import get, post

In [9]:
HEADER = {"x-dune-api-key" : DUNE_API_KEY}
BASE_URL = "https://api.dune.com/api/v1/"

In [10]:
def make_api_url(module, action, ID):
    url = BASE_URL + module + "/" + ID + "/" + action
    
    return url

def execute_query(query_id, engine="medium"):
    url = make_api_url("query", "execute", query_id)
    params = {
        "performance": engine,
    }
    response = post(url, headers=HEADER, params=params)
    execution_id = response.json()['execution_id']

    return execution_id


def get_query_status(execution_id):
    url = make_api_url("execution", "status", execution_id)
    response = get(url, headers=HEADER)

    return response


def get_query_results(execution_id):
    url = make_api_url("execution", "results", execution_id)
    response = get(url, headers=HEADER)

    return response


def cancel_query_execution(execution_id):
    url = make_api_url("execution", "cancel", execution_id)
    response = get(url, headers=HEADER)

    return response

In [11]:
execution_id = execute_query("347203","medium")

In [12]:
status = get_query_status(execution_id)
status.json()

{'execution_id': '01J6JKX60W3CYCMBMNYTV3B8CD',
 'query_id': 347203,
 'is_execution_finished': True,
 'state': 'QUERY_STATE_COMPLETED',
 'submitted_at': '2024-08-30T21:43:48.509078Z',
 'expires_at': '2024-11-28T21:43:50.841147Z',
 'execution_started_at': '2024-08-30T21:43:49.014327158Z',
 'execution_ended_at': '2024-08-30T21:43:50.841145878Z',
 'result_metadata': {'column_names': ['holder', 'holder_name', 'CVP_balance'],
  'column_types': ['varbinary', 'varchar(30)', 'double'],
  'row_count': 5779,
  'result_set_bytes': 289354,
  'total_row_count': 5779,
  'total_result_set_bytes': 289354,
  'datapoint_count': 17337,
  'pending_time_millis': 505,
  'execution_time_millis': 1826}}

Нужна задержка, пока выполнится запрос (execution ID присваивается сразу, но это не значит, что экзекьют уже выполнен)

In [13]:
results = get_query_results(execution_id)
results.json()['result']['rows']

[{'CVP_balance': 17034449,
  'holder': '0xb258302c3f209491d604165549079680708581cc',
  'holder_name': 'Multisig'},
 {'CVP_balance': 9510562,
  'holder': '0x60842a6b44cca5eb7685d1135acf69b1336b28b3',
  'holder_name': 'Old PP Vesting'},
 {'CVP_balance': 9143920,
  'holder': '0x82c2b82e71998eb94be0a9f353e7a9eb7f74c603',
  'holder_name': 'Ecosystem Fund (Grants)'},
 {'CVP_balance': 7500000,
  'holder': '0x7cf2c758d73bf08d40a05813e6e03906bb97c90a',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 5000000,
  'holder': '0x5a20e71378bc770d1cd0b9a30d96bf911f023d1c',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 5000000,
  'holder': '0x836e76c340f969e5bab521f69ea26218a189bab0',
  'holder_name': 'Team Vesting'},
 {'CVP_balance': 4836246,
  'holder': '0x0d0707963952f2fba59dd06f2b425ace40b492fe',
  'holder_name': 'Gate.io 1'},
 {'CVP_balance': 3808230,
  'holder': '0x28c6c06298d514db089934071355e5743bf21d60',
  'holder_name': 'Binance 14'},
 {'CVP_balance': 3189842,
  'holder': '0xdfd5293d8e

In [14]:
# data = pd.DataFrame(results.json()['result']['rows'])
# data